## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [5]:
import torch
import torch.nn as nn
import numpy as np

As before, we represent the inputs and targets and matrices.

In [6]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [7]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

We are using 15 training examples to illustrate how to work with large datasets in small batches. 

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [8]:
from torch.utils.data import TensorDataset

In [9]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [10]:
from torch.utils.data import DataLoader

In [11]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a `for` loop. Let's look at an example.

In [12]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 87., 135.,  57.],
        [ 73.,  67.,  43.],
        [ 68.,  96.,  71.]])
tensor([[119., 133.],
        [103., 119.],
        [118., 134.],
        [ 56.,  70.],
        [104., 118.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [13]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4104, -0.1286,  0.0376],
        [-0.4664, -0.2982,  0.1247]], requires_grad=True)
Parameter containing:
tensor([ 0.0537, -0.0974], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [14]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.4104, -0.1286,  0.0376],
         [-0.4664, -0.2982,  0.1247]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0537, -0.0974], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [15]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-36.8998, -48.7632],
        [-46.1964, -60.8021],
        [-50.6965, -73.4038],
        [-45.9395, -55.8792],
        [-37.9716, -52.1791],
        [-37.1816, -48.9313],
        [-46.0301, -60.3792],
        [-51.0692, -73.7454],
        [-45.6577, -55.7110],
        [-37.5236, -51.5880],
        [-36.7336, -48.3402],
        [-46.4781, -60.9703],
        [-50.8628, -73.8267],
        [-46.3875, -56.4703],
        [-37.6898, -52.0109]], grad_fn=<AddmmBackward0>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [16]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [17]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [18]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(19866.0430, grad_fn=<MseLossBackward0>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [19]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [20]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [21]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 190.6962
Epoch [20/100], Loss: 49.7640
Epoch [30/100], Loss: 60.2433
Epoch [40/100], Loss: 92.8213
Epoch [50/100], Loss: 18.9550
Epoch [60/100], Loss: 4.5965
Epoch [70/100], Loss: 20.1339
Epoch [80/100], Loss: 2.3991
Epoch [90/100], Loss: 12.3659
Epoch [100/100], Loss: 12.2622


Let's generate predictions using our model and verify that they're close to our targets.

In [22]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.9308,  71.2558],
        [ 81.4707,  99.8264],
        [118.2780, 133.1059],
        [ 26.0313,  42.6554],
        [ 97.8246, 114.3785],
        [ 56.7946,  70.2785],
        [ 81.1552,  99.7364],
        [118.5136, 133.6403],
        [ 27.1675,  43.6327],
        [ 98.6452, 115.2658],
        [ 57.6153,  71.1657],
        [ 80.3345,  98.8492],
        [118.5935, 133.1959],
        [ 25.2107,  41.7682],
        [ 98.9607, 115.3558]], grad_fn=<AddmmBackward0>)

In [23]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [24]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.4994, 68.4105]], grad_fn=<AddmmBackward0>)

The predicted yield of apples is 54.3 tons per hectare, and that of oranges is 68.3 tons per hectare.